In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML

In [2]:

URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'
response = requests.get(URL)
response

<Response [200]>

In [ ]:
soup = BS(response.text)
print(soup.prettify())